In [1]:
# Load the data from the Apporto machine to the Colab environment

from google.colab import files
uploaded = files.upload()

Saving PersonalLoan.csv to PersonalLoan.csv


In [2]:
# Pandas is the Python package for data frames

import pandas as pd

In [3]:
# Part 1 Data Acquisition

# Read data from a CSV file into a data frame
df = pd.read_csv('PersonalLoan.csv')

# Display the first ten rows of the data frame to examine if it is an individual-level data set
print(df.head(10))

# Display the variable list
print(df.columns.values)

# Display the number of rows and the number of columns in the data set to confirm the portrait shape
# The first element of the output is the number of rows and the second is the number of columns 
print(df.shape)

   Age  Experience  Income  ZIPCode  ...  CDAccount  Online  CreditCard  PersonalLoan
0   25           1      49    91107  ...         No      No          No            No
1   45          19      34    90089  ...         No      No          No            No
2   39          15      11    94720  ...         No      No          No            No
3   35           9     100    94112  ...         No      No          No            No
4   35           8      45    91330  ...         No      No         Yes            No
5   37          13      29    92121  ...         No     Yes          No            No
6   53          27      72    91711  ...         No     Yes          No            No
7   50          24      22    93943  ...         No      No         Yes            No
8   35          10      81    90089  ...         No     Yes          No            No
9   34           9     180    93023  ...         No      No          No           Yes

[10 rows x 13 columns]
['Age' 'Experience' 'Income' '

In [4]:
# Part 3 Missing Value Imputation

# Show the number of missing values for each variable in the data frame
df.isnull().sum()

# Drop ZIP Code for now
rvar_list =['ZIPCode']
df_sample1 = df.drop(columns=rvar_list)

# Separate all the variables into two lists for future column indexing
# One for numerical, the other for categorical 
cvar_list = ['Education', 'SecuritiesAccount', 'CDAccount', 'Online', 'CreditCard', 'PersonalLoan']
nvar_list = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Mortgage']

# Check if there is any missing value left
df_sample1.isnull().sum()


Age                  0
Experience           0
Income               0
Family               0
CCAvg                0
Education            0
Mortgage             0
SecuritiesAccount    0
CDAccount            0
Online               0
CreditCard           0
PersonalLoan         0
dtype: int64

In [5]:
# Part 4 Variable Transformation

# Standardize the numerical variables 
df_sample2 = df_sample1.copy()
df_sample2[nvar_list] = (df_sample1[nvar_list] - df_sample1[nvar_list].mean())/df_sample1[nvar_list].std()

# Set the datatype for the variables in the cvar_list to be categorical in Python
# Set the datatype for the variables in the nvar_list to be numerical in Python 
df_sample3 = df_sample2.copy()
df_sample3[cvar_list] = df_sample2[cvar_list].astype('category')
df_sample3[nvar_list] = df_sample2[nvar_list].astype('float64')

# Convert the categorical variables into dummies (Step 1 of dummy coding)
# prefix_sep is the sympol used to create the dummy variable names.

df_sample4 = df_sample3.copy()
df_sample4 = pd.get_dummies(df_sample3, prefix_sep='_')

# Remove the redundant dummies (Step 2 of dummy coding)
# Placeholder variable: rdummies
rdummies = ['Education_1', 'SecuritiesAccount_Yes', 'CDAccount_Yes', 'Online_Yes', 'CreditCard_Yes', 'PersonalLoan_No']
df_sample5 = df_sample4.copy()
df_sample5 = df_sample4.drop(columns=rdummies)

# Get the remaining variable list after the variable transformation
print(df_sample5.columns.values)

# Display the milestone dataframe. Compare it with the original dataframe.
print(df_sample5)
print(df)

['Age' 'Experience' 'Income' 'Family' 'CCAvg' 'Mortgage' 'Education_2'
 'Education_3' 'SecuritiesAccount_No' 'CDAccount_No' 'Online_No'
 'CreditCard_No' 'PersonalLoan_Yes']
           Age  Experience  ...  CreditCard_No  PersonalLoan_Yes
0    -1.774239   -1.665912  ...              1                 0
1    -0.029521   -0.096321  ...              1                 0
2    -0.552936   -0.445119  ...              1                 0
3    -0.901880   -0.968316  ...              1                 0
4    -0.901880   -1.055515  ...              0                 0
...        ...         ...  ...            ...               ...
4995 -1.425296   -1.491513  ...              1                 0
4996 -1.338060   -1.404313  ...              1                 0
4997  1.540726    1.647670  ...              1                 0
4998  1.715198    1.734869  ...              1                 0
4999 -1.512532   -1.404313  ...              0                 0

[5000 rows x 13 columns]
      Age  Experience

In [6]:
# Part 5 Data Partiton

# Required package: scikit-learn. Package name in Python: sklearn
# Required subpackage: model_selection. Required function name: train_test_split
from sklearn.model_selection import train_test_split

# Placeholder variables: df4partition, testpart_size
# test_size specifies the percentage for the test partition
df4partition = df_sample5
testpart_size = 0.2

# random_state specifies the seed for random number generator. 
# random_state = 1 unless otherwised noted
df_nontestData, df_testData = train_test_split(df4partition, test_size=testpart_size, random_state=1)

print(df_nontestData)

           Age  Experience  ...  CreditCard_No  PersonalLoan_Yes
1233  0.668367    0.775675  ...              1                 0
1056 -0.814644   -1.229914  ...              1                 0
1686  1.453490    1.560470  ...              1                 0
187   0.057715    0.078078  ...              1                 1
3840  0.930075    0.950074  ...              1                 0
...        ...         ...  ...            ...               ...
2895  1.279018    1.386071  ...              1                 0
2763  0.842839    0.950074  ...              1                 0
905   0.057715    0.165278  ...              0                 0
3980  0.057715    0.165278  ...              1                 0
235  -0.640172   -1.055515  ...              1                 0

[4000 rows x 13 columns]


In [8]:
# Part 6 Logistic Regression with Penalty 

# Required package: scikit-learn. Package name in Python: sklearn
# Required subpackage: linear_model. 
# Required function name: LogisticRegression, LogisticRegressionCV

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# Separate the predictor values and the DV values into X and y respectively
# Placeholder variable: DV
DV = 'PersonalLoan_Yes'
y = df_nontestData[DV]
X = df_nontestData.drop(columns=[DV])

# Run Logistic regression with pre-specified penalty level (alpha)
# Placeholder variable: alpha
alpha = 10

# The Logistic regression results are put into a model object clf
clf = LogisticRegression(C=1/alpha, penalty='l1', solver='saga', max_iter=200, random_state=1).fit(X,y)

# A user-defined function summary_coef
# to display the estimated coefficients of a model candidate obtained by the Logistic Regression analysis
def summary_coef(model_object):
  n_predictors = X.shape[1]
  model_coef = pd.DataFrame(model_object.coef_.reshape(1, n_predictors), columns=X.columns.values)
  model_coef['Intercept'] = model_object.intercept_
  return model_coef.transpose()

print(summary_coef(clf))



                             0
Age                   0.000000
Experience            0.000000
Income                2.275057
Family                0.586373
CCAvg                 0.192983
Mortgage              0.054617
Education_2           2.648378
Education_3           2.842589
SecuritiesAccount_No  0.000000
CDAccount_No         -1.684072
Online_No             0.117621
CreditCard_No         0.176838
Intercept            -4.386651


In [10]:
# Run Logistic regression with k-fold cross validation with k=5
# Placeholder variable: kfolds
kfolds = 5

# Here we specify within which range of the penalty levels we will search for the optimal penalty level, 
# i.e., the level that leads to the best model candidate
# We search the optimal alpha within [min_alpha, max_alpha]
min_alpha = 0.01
max_alpha = 100

# C is the inverse of alpha. LogisticRegression and LogisticRegressionCV require inverse alpha 
# as the parameter values for specifying penalty levels
max_C = 1/min_alpha
min_C = 1/max_alpha

# We further discretize the continuous C range [min_C, max_C] into n individual points of C
# We train n model candidates each of which corresponds to one individual C point
n_candidates = 5000

import numpy as np

# We store the list of individual C points into C_list
C_list = list(np.linspace(min_C, max_C, num=n_candidates))

# Create a user-defined function, profit_calcuation, that calcuates the profit over a dataset 
# for which we know the actual class (e.g., acceptance/rejection upon receiving a offer) from y_value
# and the predictors values (e.g., consumers age, experience, income) from x_value
# using the probability prediction model carried by the model object model.

def profit_calculation(model, x_value, y_value):
  
  # Specify the decision cut-off used in the decision rule
  d_cutoff = 1/11

  # Get the decisions made by the decision rule for each observation in the dataset
  # The method predict_proba is to get the predicted probability
  # Then we compare the predicted probabilities with the decision cut-off d_cutoff
  # True means SEND, False means NOT SEND
  # list() is to convert the results into a Python list
  decision = list(model.predict_proba(x_value)[:,1] > d_cutoff)
  
  # We put the actual class into a Python list called y
  y = list(y_value)
  
  # Get the number of observations of the dataset and put it into n_obs
  n_obs = len(y)

  # cum_profit is for cumulating the profit during the for-loop
  cum_profit = 0

  for i in range(n_obs): # i will go from 0 to (n_obs-1)
    if decision[i] == True and y[i] == 1: # if the decision is SEND and the actual class is 1 (Accept) for the i-th observation
      profit = 10 # the net profit is 10
    elif decision[i] == True and y[i] == 0: # if the decision is SEND and the actual class is 0 (Reject) for the i-th observation
      profit = -1 # the net profit is -1
    else:
      profit = 0 # For any other situation, the net profit is zero
    cum_profit = cum_profit + profit # cumulating the profit
  
  average_net_profit = cum_profit / n_obs # Derive the average net profit
  return average_net_profit # return the average net profit

# Set n_jobs to be -1 to run LogisticRegressionCV on all CPU cores.
# The search criterion is to find the model that maximizes 
# whatever the scoring function - for this case the profit_calculation function - returns.
clf_optimal = LogisticRegressionCV(Cs=C_list, cv=kfolds, scoring=profit_calculation, penalty='l1', solver='saga', max_iter=200, random_state=1, n_jobs=-1).fit(X,y)

# Display the estimated coefficients of the final selected model
print(summary_coef(clf_optimal))

# Display the optimal alpha that yields the final selected model (the best model candidate)
print(1/clf_optimal.C_)

# Calcuate the average net profit over the test partition based on the final selected model

# y_test_actual is the actual values of the DV in the test partition
y_test_actual = df_testData[DV]

# X_test is the predictor values in the test partition
X_test = df_testData.drop(columns=[DV])

# Use the clf_optimal object to apply the model associated with clf_optimal to the test partition
# Use the user-defined profit_calculation function to get the profit
print(profit_calculation(clf_optimal, X_test, y_test_actual))


                             0
Age                   0.000000
Experience            0.000000
Income                1.331570
Family                0.243037
CCAvg                 0.037102
Mortgage              0.000000
Education_2           0.000000
Education_3           0.000000
SecuritiesAccount_No  0.000000
CDAccount_No          0.000000
Online_No             0.000000
CreditCard_No         0.000000
Intercept            -2.977438
[100.]
0.709


In [11]:
# Part7 Score the new data

# Upload the new data file from the local drive to Colab
from google.colab import files
uploaded = files.upload()

Saving PersonalLoan_Competition_Hindsight.csv to PersonalLoan_Competition_Hindsight.csv


In [12]:
# Load the data from the new data file to a Python dataframe df_newdata
df_newdata = pd.read_csv('PersonalLoan_Competition_Hindsight.csv')

# Check if there is any regular missing value in df_newdata
print(df_newdata.isnull().sum())

Age                  0
Experience           0
Income               0
ZIPCode              0
Family               0
CCAvg                0
Education            0
Mortgage             0
SecuritiesAccount    0
CDAccount            0
Online               0
CreditCard           0
PersonalLoan         0
dtype: int64


In [15]:
# Generate the categorical predictor list
Original_DV = 'PersonalLoan'
cpredictor_list = cvar_list.copy()
cpredictor_list.remove(Original_DV)

# Drop the redundant variable, e.g., ZIPCode as we did for the historical data
df_newdata_sample1 = df_newdata.drop(columns=rvar_list)

# Set the datatypes of the variables in the new data
df_newdata_sample2 = df_newdata_sample1.copy()
df_newdata_sample2[cpredictor_list] = df_newdata_sample1[cpredictor_list].astype('category')
df_newdata_sample2[nvar_list] = df_newdata_sample1[nvar_list].astype('float64')

# Derive the sample mean and the sample standard deviation of the numerical variables in the historical data 
historical_sample_mean = df_sample1[nvar_list].mean()
historical_sample_std = df_sample1[nvar_list].std()

# Use the historical sample mean and historical sample standard deviation to standardize the new data
df_newdata_sample3 = df_newdata_sample2.copy()
df_newdata_sample3[nvar_list] = (df_newdata_sample2[nvar_list] - historical_sample_mean[nvar_list])/historical_sample_std[nvar_list]

# Code the categorical variables in the new data 
df_newdata_sample4 = pd.get_dummies(df_newdata_sample3, prefix_sep='_')

# Fix the inconsistency between the predictors of the new data and the predictors required by the final selected model 
df_newdata_sample5 = df_newdata_sample4.copy()
df_newdata_sample5 = df_newdata_sample4.drop(columns=rdummies)
df_newdata_sample6 = df_newdata_sample5.drop(columns=['PersonalLoan_Yes'])

print(df_newdata_sample6.columns.values)

['Age' 'Experience' 'Income' 'Family' 'CCAvg' 'Mortgage' 'Education_2'
 'Education_3' 'SecuritiesAccount_No' 'CDAccount_No' 'Online_No'
 'CreditCard_No']


In [16]:
# Score the new data using the model carried by the model object clf_optimal and print the average net profit
print(profit_calculation(clf_optimal, df_newdata_sample6, df_newdata_sample5['PersonalLoan_Yes']))


0.7307692307692307
